In [ ]:
import pandas as pd

added_stocks_path = r"C:\UU\THESIS\AMX\HYPOTHESIS 1\ADDED_STOCKS.xlsx"
excel_data = pd.ExcelFile(added_stocks_path)

results_list = []

for sheet_name in excel_data.sheet_names:
    df = pd.read_excel(added_stocks_path, sheet_name=sheet_name)
    
    df['Excess_Return'] = df['stock_return'] - df['market_return']
    
    df['date'] = pd.to_datetime(df['date'])
    
    df['year'] = df['date'].dt.year
  
    yearly_excess_return = df.groupby('year')['Excess_Return'].sum().reset_index()
    
    yearly_excess_return['Company'] = sheet_name
    
    results_list.append(yearly_excess_return)

excess_return_results = pd.concat(results_list, ignore_index=True)

results_path = r"C:\UU\THESIS\AMX\REGRESSION ANALYSIS\use_2005+\ADDED_Yearly_Excess_Return_Results.xlsx"
excess_return_results.to_excel(results_path, index=False)

print(excess_return_results.head())

company_dfs = {}

for company in excess_return_results['Company'].unique():
   
    company_data = excess_return_results[excess_return_results['Company'] == company]
    
    company_data = company_data.sort_values(by='year')

    company_data['event'] = range(0, min(6, len(company_data)))
    
    company_data = company_data[['event', 'year', 'Excess_Return']]
    
    company_dfs[company] = company_data

output_path = r"C:\UU\THESIS\AMX\REGRESSION ANALYSIS\use_2005+\ADDED_Transformed_Excess_Return_Results.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for company, df in company_dfs.items():
        df.to_excel(writer, sheet_name=company, index=False)



In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats.mstats import winsorize
from statsmodels.iolib.summary2 import summary_col


added_excess_returns_path = r"C:\UU\THESIS\AMX\REGRESSION ANALYSIS\use_2005+\ADDED_Transformed_Excess_Return_Results_IO_NO.xlsx"
added_proxies_path = r"C:\UU\THESIS\AMX\REGRESSION ANALYSIS\use_2005+\ADDED_PROXIES -2005+ IO_NO.xlsx"
deleted_excess_returns_path = r"C:\UU\THESIS\AMX\REGRESSION ANALYSIS\use_2005+\DELETED_Transformed_Excess_Return_Results_2005+_IO_NO.xlsx"
deleted_proxies_path = r"C:\UU\THESIS\AMX\REGRESSION ANALYSIS\use_2005+\DELETED_PROXIES IO_NO.xlsx"

added_proxies_sheets = pd.read_excel(added_proxies_path, sheet_name=None)
deleted_proxies_sheets = pd.read_excel(deleted_proxies_path, sheet_name=None)
added_excess_return_results_sheets = pd.read_excel(added_excess_returns_path, sheet_name=None)
deleted_excess_return_results_sheets = pd.read_excel(deleted_excess_returns_path, sheet_name=None)

combined_added_proxies_df = pd.concat(added_proxies_sheets.values(), ignore_index=True)
combined_deleted_proxies_df = pd.concat(deleted_proxies_sheets.values(), ignore_index=True)
combined_added_excess_return_results_df = pd.concat(added_excess_return_results_sheets.values(), ignore_index=True)
combined_deleted_excess_return_results_df = pd.concat(deleted_excess_return_results_sheets.values(), ignore_index=True)

combined_added_proxies_df['company'] = combined_added_proxies_df.index // (len(combined_added_proxies_df) // len(added_proxies_sheets))
combined_deleted_proxies_df['company'] = combined_deleted_proxies_df.index // (len(combined_deleted_proxies_df) // len(deleted_proxies_sheets))
combined_added_excess_return_results_df['company'] = combined_added_excess_return_results_df.index // (len(combined_added_excess_return_results_df) // len(added_excess_return_results_sheets))
combined_deleted_excess_return_results_df['company'] = combined_deleted_excess_return_results_df.index // (len(combined_deleted_excess_return_results_df) // len(deleted_excess_return_results_sheets))

#add a dummy variable: 0 for added, 1 for deleted
combined_added_proxies_df['deleted_stock'] = 0
combined_deleted_proxies_df['deleted_stock'] = 1

#merge the combined dataframes on the common 'event' and 'company' columns
combined_added_merged_df = pd.merge(combined_added_proxies_df, combined_added_excess_return_results_df, on=['event', 'company'])
combined_deleted_merged_df = pd.merge(combined_deleted_proxies_df, combined_deleted_excess_return_results_df, on=['event', 'company'])

combined_merged_df = pd.concat([combined_added_merged_df, combined_deleted_merged_df], ignore_index=True)

event_0_df = combined_merged_df[combined_merged_df['event'] == 0].set_index('company')
event_5_df = combined_merged_df[combined_merged_df['event'] == 5].set_index('company')

#calculate the changes
diff_df = event_5_df.copy()
diff_df['DIO_NO'] = event_5_df['IO_NO'] - event_0_df['IO_NO']
diff_df['DIlliquidity'] = event_5_df['illiquidity'] - event_0_df['illiquidity']
diff_df['DROA'] = event_5_df['roa_adjusted'] - event_0_df['roa_adjusted']
diff_df['DDispersion'] = (event_5_df['dispersion'] - event_0_df['dispersion']) / event_0_df['dispersion']
diff_df['Excess_Return'] = event_5_df['Excess_Return']  # Ensure this column is available
diff_df['deleted_stock'] = event_5_df['deleted_stock']

diff_df['company'] = event_5_df.index

#winsorize the variables at 5%
for column in [ 'DDispersion',  'DIlliquidity','DIO_NO', 'DROA', 'Excess_Return']:
    diff_df[column] = winsorize(diff_df[column], limits=[0.05, 0.05])

diff_df = diff_df.apply(pd.to_numeric, errors='coerce')

diff_df = pd.get_dummies(diff_df, columns=['year'], drop_first=True)

diff_df = diff_df.replace([float('inf'), float('-inf')], pd.NA).dropna(subset=['DDispersion', 'DIO_NO','DIlliquidity', 'DROA', 'Excess_Return', 'deleted_stock'])

#define the dependent and independent variables for the regression
X = diff_df[['DDispersion', 'DIO_NO','DIlliquidity', 'DROA','deleted_stock'] + [col for col in diff_df.columns if col.startswith('year_')]]
y = diff_df['Excess_Return']

X = X[X.index.isin(y.index)].astype(float)
y = y[y.index.isin(X.index)].astype(float)

X = sm.add_constant(X)

#perform the OLS regression and cluster standard errors by firm
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': diff_df['company']})

print("Regression Results for Combined Stocks with Dummy Variable")
print(model.summary())